In [1]:
#!pip install langchain
#!pip install langchain_community
#!pip install chromadb
#!pip install langchain-openai

In [1]:
import pandas as pd
import certifi
import urllib3
import warnings
import os
import openai

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma

import pyface
import importlib_resources
from langchain.embeddings.openai import OpenAIEmbeddings
#from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [4]:
#KB 이벤트 웹데이터
web_data = pd.read_csv('/Users/parkjungeun/DML_crawling/KB_공모전/df_emb_total_2.csv', sep=',', dtype = str)

result = [x for x in web_data['알아두세요']]
docs = result
#print(docs)

#텍스트 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splited_docs = text_splitter.create_documents(docs) #.split_documents
#print(len(splited_docs))
#print(splited_docs[10].page_content)

os.environ["OPENAI_API_KEY"] = "sk-proj-AO3KelC1emoiTrtYed3jT3BlbkFJVuDMTmSkpm2J4S7me2nO"

#임베딩
vectorstore = Chroma.from_documents(documents = splited_docs, embedding=OpenAIEmbeddings())

prompt_template = '''
Answer the question base on the following contxt:
{context}

question : {question}
'''

prompt = ChatPromptTemplate.from_template(prompt_template)
model = ChatOpenAI(model='gpt-3.5-turbo-0125', temperature=0)

retriever = vectorstore.as_retriever()

In [9]:
# Combine Documents
def format_docs(docs):
    return '\n\n'.join(doc.page_content for doc in docs)

#Chain 연결
chain = (
    {'context' : retriever | format_docs, 'question':RunnablePassthrough()}
     | prompt
     | model
     | StrOutputParser()
)

answer = chain.invoke("KB이벤트 내에서 '집합투자상품','신탁계약','예금자보호법'과 같은 어려운 금융 용어를 15개 뽑아주고 각 용어에 대한 결과를 단어 key, 해석 value json형식으로 보여줘.")
print(answer)

{
  "집합투자상품": "여러 투자자들이 자금을 모아 운용하는 상품",
  "신탁계약": "자산을 신탁사에 맡겨 운용하거나 관리하는 계약",
  "예금자보호법": "예금자의 이자와 원금을 보호하는 법",
  "예금보험공사": "은행 파산 시 예금자의 예금을 보호하는 공사",
  "보호한도": "예금자가 보호받을 수 있는 최대 금액",
  "이자": "돈을 빌려준 대가로 받는 금전적 이득",
  "이벤트": "특정 기간 동안 진행되는 행사 또는 행사 상품",
  "경품": "이벤트에서 제공되는 상품 또는 혜택",
  "상품설명서": "금융 상품의 세부 내용과 조건을 설명하는 문서",
  "약관": "금융 상품 이용 시의 규정과 조건",
  "고객센터": "고객 문의 및 서비스 제공을 위한 전화 상담 센터",
  "준법감시인": "금융기관의 법규 준수를 감독하는 기관",
  "연금": "일정 기간 동안 정기적으로 지급되는 소득",
  "수수료": "서비스나 거래에 대한 대가로 받는 금액",
  "우대금리": "특정 조건을 충족할 시 적용되는 우대 이자율"
}


In [ ]:
#file_path = '/Users/parkjungeun/Downloads/output_50.txt'

#with open(file_path, "w") as file:
    #file.write(answer)